1. Load the Dataset

2. Text Pre-Preocessing

3. Ranked Document and Evaluation Using MAP

     1. Jaccard Coefficient
   
     2. Term Frequency
   
     3. Inverse Document Frequency
        
     6. TF-IDF
      
     7. TF-IDF based Vector Space Model

In [113]:
# mounting the drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [114]:
import numpy as np
import pandas as pd
import re
import spacy

nlp = spacy.load('en_core_web_sm')

# **1. load dataset**

In [115]:
!unzip '/content/drive/MyDrive/ranked_retrival_data.zip'

Archive:  /content/drive/MyDrive/ranked_retrival_data.zip
replace dataset/documents.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: dataset/documents.csv   
  inflating: dataset/qrel.csv        
  inflating: dataset/qrel_val.csv    
  inflating: dataset/queries.csv     
  inflating: dataset/queries_val.csv  


In [116]:
#loading documents
documents = pd.read_csv('/content/dataset/documents.csv')
print("shape=>",documents.shape)
documents.head()

shape=> (387, 5)


,docid,author,bibliography,body,title
0,2,ting-yili,"department of aeronautical engineering, rensse...",simple shear flow past a flat plate in an inco...,simple shear flow past a flat plate in an inco...
1,3,m. b. glauert,"department of mathematics, university of manch...",the boundary layer in simple shear flow past a...,the boundary layer in simple shear flow past a...
2,5,"wasserman,b.","j. ae. scs. 24, 1957, 924.",one-dimensional transient heat conduction into...,one-dimensional transient heat conduction into...
3,6,"campbell,w.f.","j. ae. scs. 25, 1958, 340.",one-dimensional transient heat flow in a multi...,one-dimensional transient heat flow in a multi...
4,12,"bisplinghoff,r.l.","j. ae. scs. 23, 1956, 289.",some structural and aerelastic considerations ...,some structural and aerelastic considerations ...


In [117]:
#loading queries
queries = pd.read_csv('/content/dataset/queries.csv')
print("shape=>",queries.shape)
queries.head()

shape=> (85, 2)


,qid,query
0,1,what similarity laws must be obeyed when const...
1,2,what are the structural and aeroelastic proble...
2,3,what problems of heat conduction in composite ...
3,8,what methods -dash exact or approximate -dash ...
4,10,are real-gas transport properties for air avai...


In [118]:
#loading qrel
qrel = pd.read_csv('/content/dataset/qrel.csv')
print("shape=>",qrel.shape)
qrel.head()

shape=> (425, 2)


,qid,docid
0,1,184
1,1,29
2,1,31
3,1,57
4,1,378


In [119]:
#loading validation queries and qrel
validation_queries = pd.read_csv('/content/dataset/queries_val.csv')
print("shape=>",validation_queries.shape)
validation_queries.head()

shape=> (22, 2)


,qid,query
0,189,is there a design method for calculating therm...
1,190,will an analysis of panel flutter based on arb...
2,191,"what is the criterion for true panel flutter, ..."
3,194,how can the analytical solution of the bucklin...
4,196,the problem of similarity for representative i...


In [120]:
#loading validation qrel
validation_qrel = pd.read_csv('/content/dataset/qrel_val.csv')
print("shape=>",validation_qrel.shape)
validation_qrel.head()

shape=> (110, 2)


,qid,docid
0,189,395
1,189,866
2,189,869
3,189,865
4,189,868


**reading the sample**

In [121]:
queries['query'].sample(10).values

array(['are there any theoretical methods for predicting base pressure .',
       'jet interference with supersonic flow  -dash experimental papers .',
       'what agreement is found between theoretically predicted instability times and experimentally measured collapse times for compressed columns in creep .',
       'what are the best experimental data and classical small deflection theory analyses available for pressurized cylinders in bending .',
       'can the load deformation characteristics of a beam be obtained with the material being inelastic and a non uniform temperature being present .',
       'recent data on shock-induced boundary-layer separation .',
       'has anyone derived simplified pump design equation from the fundamental three-dimensional equations for incompressible nonviscous flow .',
       'what information is available for dynamic response of airplanes to gusts or blasts in the subsonic regime .',
       'how can one detect transition phenomena in hypersoni

In [122]:
documents['body'][10:15].values

array(['a simple model study of transient temperature and thermal stress distribution due to aerodynamic heating .   the present work is concerned with the determination of transient temperatures and thermal stresses in simple models intended to simulate parts or the whole of an aircraft structure of the built- up variety subjected to aerodynamic heating .   the first case considered is that of convective heat transfer into one side of a flat plate, representing a thick skin, and the effect of the resulting temperature distribution in inducing thermal stresses associated with bending restraint at the plate edges . numerical results are presented for the transient temperature differentials in the plate when the environment temperature first increases linearly with time and then remains constant, the period of linear increase representing the time of acceleration of the aircraft .  corresponding thermal stress information is presented .   the second case is that of the wide-flanged i-bea

#**2. text processsing**

In [123]:
def preprocess(text):
    text = re.sub("-"," ",text)
    text = re.sub("[^a-z]+"," ",text)
    text = re.sub("[\s]+"," ",text)
    doc = nlp(text)

    tokens = [token.lemma_ for token in doc if(token.is_stop==False)]
    return tokens

In [124]:
documents['tokens']=documents['body'].apply(preprocess)
documents.head()

,docid,author,bibliography,body,title,tokens
0,2,ting-yili,"department of aeronautical engineering, rensse...",simple shear flow past a flat plate in an inco...,simple shear flow past a flat plate in an inco...,"[simple, shear, flow, past, flat, plate, incom..."
1,3,m. b. glauert,"department of mathematics, university of manch...",the boundary layer in simple shear flow past a...,the boundary layer in simple shear flow past a...,"[boundary, layer, simple, shear, flow, past, f..."
2,5,"wasserman,b.","j. ae. scs. 24, 1957, 924.",one-dimensional transient heat conduction into...,one-dimensional transient heat conduction into...,"[dimensional, transient, heat, conduction, dou..."
3,6,"campbell,w.f.","j. ae. scs. 25, 1958, 340.",one-dimensional transient heat flow in a multi...,one-dimensional transient heat flow in a multi...,"[dimensional, transient, heat, flow, multilaye..."
4,12,"bisplinghoff,r.l.","j. ae. scs. 23, 1956, 289.",some structural and aerelastic considerations ...,some structural and aerelastic considerations ...,"[structural, aerelastic, consideration, high, ..."


In [125]:
queries['tokens']=queries['query'].apply(preprocess)
queries.head()

,qid,query,tokens
0,1,what similarity laws must be obeyed when const...,"[similarity, law, obey, construct, aeroelastic..."
1,2,what are the structural and aeroelastic proble...,"[structural, aeroelastic, problem, associate, ..."
2,3,what problems of heat conduction in composite ...,"[problem, heat, conduction, composite, slab, s..."
3,8,what methods -dash exact or approximate -dash ...,"[method, dash, exact, approximate, dash, prese..."
4,10,are real-gas transport properties for air avai...,"[real, gas, transport, property, air, availabl..."


In [126]:
validation_queries['tokens']=validation_queries['query'].apply(preprocess)
validation_queries.head()

,qid,query,tokens
0,189,is there a design method for calculating therm...,"[design, method, calculate, thermal, fatigue, ..."
1,190,will an analysis of panel flutter based on arb...,"[analysis, panel, flutter, base, arbitrarily, ..."
2,191,"what is the criterion for true panel flutter, ...","[criterion, true, panel, flutter, oppose, smal..."
3,194,how can the analytical solution of the bucklin...,"[analytical, solution, buckle, strength, unifo..."
4,196,the problem of similarity for representative i...,"[problem, similarity, representative, investig..."


#**3. ranking documents and Evaluating using MAP**

#**JACCARD Coefficient**

In [127]:
temp_doc= documents[['docid', 'tokens']].copy()


In [128]:
def jaccard_coefficient(dtokens,qtokens):
  #calculating |a intersection b|
  numerator = len(set(dtokens).intersection(set(qtokens)))
  #calculating |a union b|
  denominator = len(set(dtokens).union(set(qtokens)))
  return numerator/denominator

In [129]:
# calculating jaccard coeff. for simple query document pair
jaccard_coefficient(temp_doc['tokens'][0],queries['tokens'][0])

0.02666666666666667

In [130]:
# apply it to all the values
temp_doc['jaccard_coeff']=temp_doc['tokens'].apply(lambda x:jaccard_coefficient(x,queries['tokens'][0]))
temp_doc.head()

,docid,tokens,jaccard_coeff
0,2,"[simple, shear, flow, past, flat, plate, incom...",0.026667
1,3,"[boundary, layer, simple, shear, flow, past, f...",0.000000
2,5,"[dimensional, transient, heat, conduction, dou...",0.028571
3,6,"[dimensional, transient, heat, flow, multilaye...",0.020408
4,12,"[structural, aerelastic, consideration, high, ...",0.084746


In [131]:
# doid of top 10 most relevant docs
temp_doc.sort_values(by='jaccard_coeff',ascending=False).head(10).reset_index(drop=True)

,docid,tokens,jaccard_coeff
0,12,"[structural, aerelastic, consideration, high, ...",0.084746
1,51,"[theory, aircraft, structural, model, subject,...",0.084746
2,378,"[engineering, relation, friction, heat, transf...",0.073171
3,670,"[blunt, body, heat, transfer, hypersonic, spee...",0.066667
4,875,"[model, aeroelastic, investigation, addendum, ...",0.066667
5,184,"[scale, model, thermo, aeroelastic, research, ...",0.057971
6,1111,"[research, high, speed, flutter, paper, presen...",0.057143
7,629,"[second, order, effect, laminar, boundary, lay...",0.055556
8,1305,"[propose, programme, wind, tunnel, test, hyper...",0.055556
9,84,"[experimental, investigation, downstream, infl...",0.053333


In [132]:
# doid of top 5 most relevant docs
temp_doc.sort_values(by='jaccard_coeff',ascending=False).head(5)['docid'].values

array([ 12,  51, 378, 670, 875])

In [133]:
def jaccard_rank(qtokens):
  temp_doc['jaccard_coeff']=temp_doc['tokens'].apply(lambda x:jaccard_coefficient(x,qtokens))
  relevent_docids=temp_doc.sort_values(by='jaccard_coeff',ascending=False).head()['docid'].values
  return relevent_docids

In [134]:
queries['jaccard_rel']=queries['tokens'].apply(lambda x: jaccard_rank(x))
queries.head()

,qid,query,tokens,jaccard_rel
0,1,what similarity laws must be obeyed when const...,"[similarity, law, obey, construct, aeroelastic...","[12, 51, 378, 670, 875]"
1,2,what are the structural and aeroelastic proble...,"[structural, aeroelastic, problem, associate, ...","[12, 51, 700, 746, 875]"
2,3,what problems of heat conduction in composite ...,"[problem, heat, conduction, composite, slab, s...","[5, 584, 6, 145, 582]"
3,8,what methods -dash exact or approximate -dash ...,"[method, dash, exact, approximate, dash, prese...","[122, 1306, 639, 655, 988]"
4,10,are real-gas transport properties for air avai...,"[real, gas, transport, property, air, availabl...","[405, 302, 436, 583, 1009]"


In [135]:
#evaluating of train set
#adding ground truth in the column
queries['ground_truth']=queries['qid'].apply(lambda x:qrel[qrel['qid']==x]['docid'].values)
queries.head()

,qid,query,tokens,jaccard_rel,ground_truth
0,1,what similarity laws must be obeyed when const...,"[similarity, law, obey, construct, aeroelastic...","[12, 51, 378, 670, 875]","[184, 29, 31, 57, 378]"
1,2,what are the structural and aeroelastic proble...,"[structural, aeroelastic, problem, associate, ...","[12, 51, 700, 746, 875]","[12, 746, 15, 184, 858]"
2,3,what problems of heat conduction in composite ...,"[problem, heat, conduction, composite, slab, s...","[5, 584, 6, 145, 582]","[5, 6, 90, 91, 119]"
3,8,what methods -dash exact or approximate -dash ...,"[method, dash, exact, approximate, dash, prese...","[122, 1306, 639, 655, 988]","[48, 122, 354, 360, 1005]"
4,10,are real-gas transport properties for air avai...,"[real, gas, transport, property, air, availabl...","[405, 302, 436, 583, 1009]","[259, 405, 302, 436, 437]"


In [136]:
def avrage_precision(model_rel,ground_truth):
  tp=0
  precisions=[]

  #finding precision at position at which relevant doc to returned
  for index,rel in enumerate(model_rel):
    if rel in ground_truth:
      tp+=1
      precisions.append(tp/(index+1))

  # if no relvent doc in list then return 0
  if precisions==[]:
    return 0
  return np.mean(precisions)

In [137]:
# finding avg precision for each query
queries['avg_precision']=queries.apply(lambda x:avrage_precision(x['jaccard_rel'],x['ground_truth']),axis=1)
queries.head()

,qid,query,tokens,jaccard_rel,ground_truth,avg_precision
0,1,what similarity laws must be obeyed when const...,"[similarity, law, obey, construct, aeroelastic...","[12, 51, 378, 670, 875]","[184, 29, 31, 57, 378]",0.333333
1,2,what are the structural and aeroelastic proble...,"[structural, aeroelastic, problem, associate, ...","[12, 51, 700, 746, 875]","[12, 746, 15, 184, 858]",0.750000
2,3,what problems of heat conduction in composite ...,"[problem, heat, conduction, composite, slab, s...","[5, 584, 6, 145, 582]","[5, 6, 90, 91, 119]",0.833333
3,8,what methods -dash exact or approximate -dash ...,"[method, dash, exact, approximate, dash, prese...","[122, 1306, 639, 655, 988]","[48, 122, 354, 360, 1005]",1.000000
4,10,are real-gas transport properties for air avai...,"[real, gas, transport, property, air, availabl...","[405, 302, 436, 583, 1009]","[259, 405, 302, 436, 437]",1.000000


In [138]:
print("mean average precision =>",queries['avg_precision'].mean())

mean average precision => 0.49843137254901954


evaluation on validation set

In [139]:
validation_queries['ground_truth']=validation_queries['qid'].apply(lambda x: validation_qrel[validation_qrel['qid']==x]['docid'].values)


In [140]:
validation_queries['jaccard_rel']=validation_queries['tokens'].apply(lambda x: jaccard_rank(x))
validation_queries.head()


,qid,query,tokens,ground_truth,jaccard_rel
0,189,is there a design method for calculating therm...,"[design, method, calculate, thermal, fatigue, ...","[395, 866, 869, 865, 868]","[868, 1306, 833, 906, 909]"
1,190,will an analysis of panel flutter based on arb...,"[analysis, panel, flutter, base, arbitrarily, ...","[15, 391, 285, 390, 864]","[390, 1008, 285, 391, 1014]"
2,191,"what is the criterion for true panel flutter, ...","[criterion, true, panel, flutter, oppose, smal...","[914, 915, 285, 857, 858]","[285, 31, 864, 728, 15]"
3,194,how can the analytical solution of the bucklin...,"[analytical, solution, buckle, strength, unifo...","[739, 740, 742, 743, 744]","[932, 744, 1050, 1172, 938]"
4,196,the problem of similarity for representative i...,"[problem, similarity, representative, investig...","[51, 185, 874, 875, 876]","[875, 1008, 184, 864, 655]"


In [141]:
validation_queries['avg_precision']=validation_queries.apply(lambda x: avrage_precision(x['jaccard_rel'],x['ground_truth']),axis=1)
validation_queries.head()


,qid,query,tokens,ground_truth,jaccard_rel,avg_precision
0,189,is there a design method for calculating therm...,"[design, method, calculate, thermal, fatigue, ...","[395, 866, 869, 865, 868]","[868, 1306, 833, 906, 909]",1.000000
1,190,will an analysis of panel flutter based on arb...,"[analysis, panel, flutter, base, arbitrarily, ...","[15, 391, 285, 390, 864]","[390, 1008, 285, 391, 1014]",0.805556
2,191,"what is the criterion for true panel flutter, ...","[criterion, true, panel, flutter, oppose, smal...","[914, 915, 285, 857, 858]","[285, 31, 864, 728, 15]",1.000000
3,194,how can the analytical solution of the bucklin...,"[analytical, solution, buckle, strength, unifo...","[739, 740, 742, 743, 744]","[932, 744, 1050, 1172, 938]",0.500000
4,196,the problem of similarity for representative i...,"[problem, similarity, representative, investig...","[51, 185, 874, 875, 876]","[875, 1008, 184, 864, 655]",1.000000


In [142]:
print("mean average precision on validation set =>",validation_queries['avg_precision'].mean())

mean average precision on validation set => 0.4454545454545454


#**term frequency (tf)**

In [143]:
vocabulary = set()

#creating vocab
for i in documents['tokens'].values:
  vocabulary = vocabulary.union(set(i))
#sorting the vocabulary alphabetically
vocabulary = sorted(vocabulary)

print("length of vocabulary =>",len(vocabulary))

length of vocabulary => 2998


In [144]:
tf_list_doc = []
#getting term frequencies
for tokens in documents['tokens']:
  #initializing a dictionary with 0 frequency
  doc_dict=dict.fromkeys(vocabulary,0)

  #counting term frequencies
  for term in tokens:
    doc_dict[term]+=1
  #adding dict to the list
  tf_list_doc.append(doc_dict)

In [145]:
len(tf_list_doc)

387

In [146]:
#creating a dataframe of term frequencies for documents
documents_tf=pd.concat([documents['docid'],pd.DataFrame(tf_list_doc)],axis=1)
print("shape=>",documents.shape)
documents_tf.head()

shape=> (387, 6)


,docid,ab,abbreviate,ability,ablate,ablating,ablation,able,abrupt,abruptly,...,year,yield,york,young,z,zbrozek,zero,zeroth,zone,zuk
0,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,6,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,12,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [147]:
# getting term frequency for the first document in the dataset
print('Document ID:',documents_tf['docid'][0])
documents_tf.loc[0,documents_tf.loc[0,:]!=0][1:].to_dict()

Document ID: 2


{'approximation': 1,
 'arise': 1,
 'body': 2,
 'boundary': 5,
 'classical': 1,
 'consequently': 1,
 'consider': 2,
 'constant': 1,
 'curved': 1,
 'different': 1,
 'dimensional': 2,
 'discuss': 1,
 'discussion': 1,
 'edge': 1,
 'effect': 1,
 'emit': 1,
 'exist': 1,
 'feature': 1,
 'ferri': 1,
 'flat': 3,
 'flow': 6,
 'fluid': 2,
 'free': 3,
 'high': 1,
 'hypersonic': 2,
 'incompressible': 2,
 'instance': 1,
 'investigate': 1,
 'inviscid': 3,
 'irrotational': 1,
 'layer': 5,
 'leading': 1,
 'libby': 1,
 'necessary': 1,
 'nose': 1,
 'novel': 1,
 'original': 1,
 'outside': 1,
 'paper': 1,
 'past': 4,
 'plate': 3,
 'possible': 1,
 'prandtl': 2,
 'present': 1,
 'problem': 4,
 'recently': 1,
 'region': 1,
 'restrict': 1,
 'rotational': 2,
 's': 2,
 'shear': 2,
 'shock': 2,
 'show': 1,
 'simple': 2,
 'situation': 2,
 'small': 2,
 'somewhat': 1,
 'speed': 1,
 'steady': 1,
 'stream': 3,
 'study': 2,
 'treat': 1,
 'usually': 1,
 'viscosity': 2,
 'viscous': 2,
 'vorticity': 2,
 'wave': 2}

#**Log Normalizatin Term Frequency,**

In [148]:
def log_normalize(x):
  if x!=0:
    return 1+np.log10(x)
  return 0

In [149]:
#log normalizing the term counts
documents_tf.iloc[:,1:]=documents_tf.iloc[:,1:].applymap(log_normalize)

<ipython-input-149-8f958197a974>:2: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  documents_tf.iloc[:,1:]=documents_tf.iloc[:,1:].applymap(log_normalize)


In [150]:
#log normalize terms for first doc in the dataset
print('Document ID:',documents_tf['docid'][0])
documents_tf.loc[0,documents_tf.loc[0,:]!=0][1:].to_dict()

Document ID: 2


{'approximation': 1.0,
 'arise': 1.0,
 'body': 1.3010299956639813,
 'boundary': 1.6989700043360187,
 'classical': 1.0,
 'consequently': 1.0,
 'consider': 1.3010299956639813,
 'constant': 1.0,
 'curved': 1.0,
 'different': 1.0,
 'dimensional': 1.3010299956639813,
 'discuss': 1.0,
 'discussion': 1.0,
 'edge': 1.0,
 'effect': 1.0,
 'emit': 1.0,
 'exist': 1.0,
 'feature': 1.0,
 'ferri': 1.0,
 'flat': 1.4771212547196624,
 'flow': 1.7781512503836436,
 'fluid': 1.3010299956639813,
 'free': 1.4771212547196624,
 'high': 1.0,
 'hypersonic': 1.3010299956639813,
 'incompressible': 1.3010299956639813,
 'instance': 1.0,
 'investigate': 1.0,
 'inviscid': 1.4771212547196624,
 'irrotational': 1.0,
 'layer': 1.6989700043360187,
 'leading': 1.0,
 'libby': 1.0,
 'necessary': 1.0,
 'nose': 1.0,
 'novel': 1.0,
 'original': 1.0,
 'outside': 1.0,
 'paper': 1.0,
 'past': 1.6020599913279625,
 'plate': 1.4771212547196624,
 'possible': 1.0,
 'prandtl': 1.3010299956639813,
 'present': 1.0,
 'problem': 1.6020599913

**Ranking**

In [151]:
# getting a list of unique query terms which are present in vocabulary
qtokens = list(set(queries['tokens'][0]).intersection(vocabulary))
print('token in query: ',len(queries['tokens'][0]),queries['tokens'][0])
print('token in vocabulary: ',len(qtokens),qtokens)

token in query:  10 ['similarity', 'law', 'obey', 'construct', 'aeroelastic', 'model', 'heat', 'high', 'speed', 'aircraft']
token in vocabulary:  9 ['similarity', 'high', 'construct', 'law', 'heat', 'speed', 'model', 'aeroelastic', 'aircraft']


In [152]:
print('tokens not in wocabulary', set(queries['tokens'][0]).difference(set(qtokens)))

tokens not in wocabulary {'obey'}


In [153]:
#creating list of column to retrieve

column=['docid']
column.extend(qtokens)
column

['docid',
 'similarity',
 'high',
 'construct',
 'law',
 'heat',
 'speed',
 'model',
 'aeroelastic',
 'aircraft']

In [154]:
#retreving terms frequency for query terms
temp_doc=documents_tf.loc[:,column].copy()
temp_doc.head()

,docid,similarity,high,construct,law,heat,speed,model,aeroelastic,aircraft
0,2,0.0,1.00000,0.0,0.0,0.000000,1.00000,0.0,0.00000,0.00000
1,3,0.0,0.00000,0.0,0.0,0.000000,0.00000,0.0,0.00000,0.00000
2,5,0.0,0.00000,0.0,0.0,1.602060,0.00000,0.0,0.00000,0.00000
3,6,0.0,0.00000,0.0,0.0,1.477121,0.00000,0.0,0.00000,0.00000
4,12,0.0,1.60206,0.0,0.0,1.000000,1.60206,0.0,1.30103,1.30103


In [155]:
#adding all the frequencies
temp_doc['tf_sum']=temp_doc[qtokens].sum(axis=1)
temp_doc.head()

,docid,similarity,high,construct,law,heat,speed,model,aeroelastic,aircraft,tf_sum
0,2,0.0,1.00000,0.0,0.0,0.000000,1.00000,0.0,0.00000,0.00000,2.000000
1,3,0.0,0.00000,0.0,0.0,0.000000,0.00000,0.0,0.00000,0.00000,0.000000
2,5,0.0,0.00000,0.0,0.0,1.602060,0.00000,0.0,0.00000,0.00000,1.602060
3,6,0.0,0.00000,0.0,0.0,1.477121,0.00000,0.0,0.00000,0.00000,1.477121
4,12,0.0,1.60206,0.0,0.0,1.000000,1.60206,0.0,1.30103,1.30103,6.806180


In [156]:
#sorting dataframe according to sum of tf and getting relevant docs
temp_doc.sort_values(by='tf_sum',ascending=False).head()['docid'].values

array([ 51,  12, 184, 364, 572])

In [157]:
def tf_rank(qtokens):
  #getting a lsit of unique query terms which are present in vocab
  qtokens = list(set(qtokens).intersection(vocabulary))
  #creating list of cols to retrive
  columns=['docid']
  columns.extend(qtokens)
  #retreving terms frequency for query terms
  temp_doc=documents_tf.loc[:,columns].copy()
  #adding all the frequencies
  temp_doc['tf_sum']=temp_doc[qtokens].sum(axis=1)
  #sorting dataframe according to sum of tf and getting relevant docs
  relevent_docs=temp_doc.sort_values(by='tf_sum',ascending=False).head()['docid'].values
  return relevent_docs

In [158]:
#ranking docs according to term frequency
queries['tf_rel']=queries['tokens'].apply(lambda x: tf_rank(x))
queries.head()

,qid,query,tokens,jaccard_rel,ground_truth,avg_precision,tf_rel
0,1,what similarity laws must be obeyed when const...,"[similarity, law, obey, construct, aeroelastic...","[12, 51, 378, 670, 875]","[184, 29, 31, 57, 378]",0.333333,"[51, 12, 184, 364, 572]"
1,2,what are the structural and aeroelastic proble...,"[structural, aeroelastic, problem, associate, ...","[12, 51, 700, 746, 875]","[12, 746, 15, 184, 858]",0.750000,"[12, 172, 51, 746, 798]"
2,3,what problems of heat conduction in composite ...,"[problem, heat, conduction, composite, slab, s...","[5, 584, 6, 145, 582]","[5, 6, 90, 91, 119]",0.833333,"[5, 980, 584, 91, 395]"
3,8,what methods -dash exact or approximate -dash ...,"[method, dash, exact, approximate, dash, prese...","[122, 1306, 639, 655, 988]","[48, 122, 354, 360, 1005]",1.000000,"[122, 234, 1104, 924, 1307]"
4,10,are real-gas transport properties for air avai...,"[real, gas, transport, property, air, availabl...","[405, 302, 436, 583, 1009]","[259, 405, 302, 436, 437]",1.000000,"[302, 185, 616, 1009, 1313]"


**evaluation on train set**

In [159]:
queries['tf_ap']=queries.apply(lambda x: avrage_precision(x['tf_rel'],x['ground_truth']),axis=1)
queries.head()

,qid,query,tokens,jaccard_rel,ground_truth,avg_precision,tf_rel,tf_ap
0,1,what similarity laws must be obeyed when const...,"[similarity, law, obey, construct, aeroelastic...","[12, 51, 378, 670, 875]","[184, 29, 31, 57, 378]",0.333333,"[51, 12, 184, 364, 572]",0.333333
1,2,what are the structural and aeroelastic proble...,"[structural, aeroelastic, problem, associate, ...","[12, 51, 700, 746, 875]","[12, 746, 15, 184, 858]",0.750000,"[12, 172, 51, 746, 798]",0.750000
2,3,what problems of heat conduction in composite ...,"[problem, heat, conduction, composite, slab, s...","[5, 584, 6, 145, 582]","[5, 6, 90, 91, 119]",0.833333,"[5, 980, 584, 91, 395]",0.750000
3,8,what methods -dash exact or approximate -dash ...,"[method, dash, exact, approximate, dash, prese...","[122, 1306, 639, 655, 988]","[48, 122, 354, 360, 1005]",1.000000,"[122, 234, 1104, 924, 1307]",1.000000
4,10,are real-gas transport properties for air avai...,"[real, gas, transport, property, air, availabl...","[405, 302, 436, 583, 1009]","[259, 405, 302, 436, 437]",1.000000,"[302, 185, 616, 1009, 1313]",1.000000


In [160]:
#finding mean avrage precision
print("mean average precision =>",queries['tf_ap'].mean())

mean average precision => 0.5886274509803923


**Evaluation on validation set**

In [161]:
#ranking documents according to term frequency
validation_queries['tf_rel']=validation_queries['tokens'].apply(lambda x: tf_rank(x))

#Finding average precision for each query
validation_queries['tf_ap']=validation_queries.apply(lambda x: avrage_precision(x['tf_rel'],x['ground_truth']),axis=1)

#finding mean avrage precision
print("mean average precision =>",validation_queries['tf_ap'].mean())

mean average precision => 0.46641414141414134


#**Inverse Document Frequency**



In [162]:
print('No. of Documents=>',documents.shape[0])

No. of Documents=> 387


In [163]:
# Term Frequency of documents
documents_tf.head()

,docid,ab,abbreviate,ability,ablate,ablating,ablation,able,abrupt,abruptly,...,year,yield,york,young,z,zbrozek,zero,zeroth,zone,zuk
0,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,12,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [164]:
# Initializing a dictionary to stores IDF values
idf_dict=dict.fromkeys(vocabulary,0)

# Count of non-zero values per column
non_zero_count=np.count_nonzero(documents_tf.iloc[:,1:],axis=0)

# Assigning IDF values
for term,document_frequency in zip(list(vocabulary),non_zero_count):
  idf_dict[term]=np.log10(documents.shape[0]/(document_frequency))

In [165]:
idf_dict

{'ab': 2.5877109650189114,
 'abbreviate': 2.5877109650189114,
 'ability': 2.5877109650189114,
 'ablate': 2.28668096935493,
 'ablating': 2.5877109650189114,
 'ablation': 1.985650973690949,
 'able': 2.28668096935493,
 'abrupt': 2.5877109650189114,
 'abruptly': 2.28668096935493,
 'absence': 2.28668096935493,
 'absolute': 1.8887409606828927,
 'absorb': 2.5877109650189114,
 'absorption': 2.5877109650189114,
 'academic': 2.5877109650189114,
 'accelerate': 2.28668096935493,
 'accelerated': 2.28668096935493,
 'acceleration': 2.110589710299249,
 'accept': 2.110589710299249,
 'acceptability': 2.5877109650189114,
 'acceptance': 2.5877109650189114,
 'accessible': 2.5877109650189114,
 'accidental': 2.5877109650189114,
 'accommodate': 2.5877109650189114,
 'accommodation': 2.5877109650189114,
 'accompany': 2.28668096935493,
 'accompanying': 2.5877109650189114,
 'accomplish': 2.28668096935493,
 'accord': 1.7426129250046545,
 'accordance': 2.5877109650189114,
 'accordingly': 1.985650973690949,
 'accoun

**Ranking**

In [166]:
# Creating a temporary dataframe
temp_doc=documents[['docid','tokens']].copy()

In [167]:
# Function for getting sum of IDF values for a query-document pair
def idf_sum(dtokens,qtokens):
  # Getting common terms in query and document
  common_term=set(dtokens).intersection(set(qtokens))

  # Getting IDF values for common terms
  idf_list=[ value for key,value in idf_dict.items() if key in common_term]

  return sum(idf_list)

In [168]:
def idf_rank(qtokens):
  # Getting sum of IDF values for all the quer-document pairs
  temp_doc['idf_sum']=temp_doc['tokens'].apply(lambda x: idf_sum(x,qtokens))

  # Sorting dataframe according to sum of IDF and getting relevant docs
  rel_docs=temp_doc.sort_values(by='idf_sum',ascending=False).head()['docid'].values

  return rel_docs

In [169]:
# Ranking documents according to inverse document frequency
queries['idf_rel']=queries['tokens'].apply(lambda x: idf_rank(x))
queries.head()

,qid,query,tokens,jaccard_rel,ground_truth,avg_precision,tf_rel,tf_ap,idf_rel
0,1,what similarity laws must be obeyed when const...,"[similarity, law, obey, construct, aeroelastic...","[12, 51, 378, 670, 875]","[184, 29, 31, 57, 378]",0.333333,"[51, 12, 184, 364, 572]",0.333333,"[184, 51, 12, 625, 332]"
1,2,what are the structural and aeroelastic proble...,"[structural, aeroelastic, problem, associate, ...","[12, 51, 700, 746, 875]","[12, 746, 15, 184, 858]",0.750000,"[12, 172, 51, 746, 798]",0.750000,"[12, 172, 51, 746, 364]"
2,3,what problems of heat conduction in composite ...,"[problem, heat, conduction, composite, slab, s...","[5, 584, 6, 145, 582]","[5, 6, 90, 91, 119]",0.833333,"[5, 980, 584, 91, 395]",0.750000,"[5, 91, 625, 584, 90]"
3,8,what methods -dash exact or approximate -dash ...,"[method, dash, exact, approximate, dash, prese...","[122, 1306, 639, 655, 988]","[48, 122, 354, 360, 1005]",1.000000,"[122, 234, 1104, 924, 1307]",1.000000,"[122, 556, 1104, 924, 234]"
4,10,are real-gas transport properties for air avai...,"[real, gas, transport, property, air, availabl...","[405, 302, 436, 583, 1009]","[259, 405, 302, 436, 437]",1.000000,"[302, 185, 616, 1009, 1313]",1.000000,"[302, 332, 405, 1009, 583]"


**Evaluation on Train set**

In [172]:
queries['idf_ap']=queries.apply(lambda x: avrage_precision(x['idf_rel'],x['ground_truth']),axis=1)

# Finding Mean Average Precision
print('Mean Average Precision=>',queries['idf_ap'].mean())

Mean Average Precision=> 0.6402614379084969


**Evaluation on validation set**

In [173]:
# Ranking documents according to inverse document frequency
validation_queries['idf_rel']=validation_queries['tokens'].apply(lambda x: idf_rank(x))

# Finding average precision for each query
validation_queries['idf_ap']=validation_queries.apply(lambda x: avrage_precision(x['idf_rel'],x['ground_truth']),axis=1)

# Finding Mean Average Precision
print('Mean Average Precision on Validation Set=>',validation_queries['idf_ap'].mean())

Mean Average Precision on Validation Set=> 0.380050505050505


#**TF-IDF**

In [174]:
documents_tfidf=documents_tf.iloc[:,1:]*list(idf_dict.values())
documents_tfidf.head()

,ab,abbreviate,ability,ablate,ablating,ablation,able,abrupt,abruptly,absence,...,year,yield,york,young,z,zbrozek,zero,zeroth,zone,zuk
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [175]:
# Adding a column of docids
documents_tfidf=pd.concat([documents['docid'],documents_tfidf],axis=1)
print('Shape=>',documents_tfidf.shape)
documents_tfidf.head()

Shape=> (387, 2999)


,docid,ab,abbreviate,ability,ablate,ablating,ablation,able,abrupt,abruptly,...,year,yield,york,young,z,zbrozek,zero,zeroth,zone,zuk
0,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,12,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [176]:
# Getting TF_IDF values for the terms in first document
print('Document ID:',documents_tfidf['docid'][0])
documents_tfidf.loc[0,documents_tfidf.loc[0,:]!=0][1:].to_dict()

Document ID: 2


{'approximation': 0.9249531333373373,
 'arise': 1.4415829293406734,
 'body': 0.8179179714411188,
 'boundary': 0.7825560609575195,
 'classical': 1.5463182798606863,
 'consequently': 2.110589710299249,
 'consider': 1.0167952618775644,
 'constant': 0.8887409606828927,
 'curved': 1.6334684555795866,
 'different': 1.1405529336766922,
 'dimensional': 1.0532614720216564,
 'discuss': 0.7243881048984555,
 'discussion': 1.2074997233073055,
 'edge': 0.7488618742816561,
 'effect': 0.43237492755384954,
 'emit': 2.5877109650189114,
 'exist': 1.0314084642516241,
 'feature': 1.3835909823629866,
 'ferri': 2.110589710299249,
 'flat': 1.1061597913506378,
 'flow': 0.5293135804637596,
 'fluid': 1.3741895767992272,
 'free': 1.1444700154382332,
 'high': 0.5964848893264165,
 'hypersonic': 1.1024255232317401,
 'incompressible': 1.311343739741794,
 'instance': 1.8887409606828927,
 'investigate': 1.0195092409519164,
 'inviscid': 1.8692846438486488,
 'irrotational': 1.985650973690949,
 'layer': 0.8049169630236986

**Ranking**

In [177]:
def tf_idf_rank(qtokens):
  # Getting a list of unique query terms which are present in vocabulary
  qtokens=list(set(qtokens).intersection(vocabulary))

  # Creating list of columns to retrieve
  columns=['docid']
  columns.extend(qtokens)

  # Retireving TF-IDF for query terms
  temp_doc=documents_tfidf.loc[:,columns].copy()

  # Adding all the frequencies
  temp_doc['tfidf_sum']=temp_doc[qtokens].sum(axis=1)

  # Sorting dataframe according to sum of TF-IDF and getting relevant docs
  rel_docs=temp_doc.sort_values(by='tfidf_sum',ascending=False).head()['docid'].values

  return rel_docs

In [178]:
# Ranking documents according to TF-IDF
queries['tfidf_rel']=queries['tokens'].apply(lambda x: tf_idf_rank(x))
queries.head()

,qid,query,tokens,jaccard_rel,ground_truth,avg_precision,tf_rel,tf_ap,idf_rel,idf_ap,tfidf_rel
0,1,what similarity laws must be obeyed when const...,"[similarity, law, obey, construct, aeroelastic...","[12, 51, 378, 670, 875]","[184, 29, 31, 57, 378]",0.333333,"[51, 12, 184, 364, 572]",0.333333,"[184, 51, 12, 625, 332]",1.000000,"[51, 184, 12, 332, 625]"
1,2,what are the structural and aeroelastic proble...,"[structural, aeroelastic, problem, associate, ...","[12, 51, 700, 746, 875]","[12, 746, 15, 184, 858]",0.750000,"[12, 172, 51, 746, 798]",0.750000,"[12, 172, 51, 746, 364]",0.750000,"[12, 51, 172, 746, 724]"
2,3,what problems of heat conduction in composite ...,"[problem, heat, conduction, composite, slab, s...","[5, 584, 6, 145, 582]","[5, 6, 90, 91, 119]",0.833333,"[5, 980, 584, 91, 395]",0.750000,"[5, 91, 625, 584, 90]",0.866667,"[5, 91, 90, 584, 625]"
3,8,what methods -dash exact or approximate -dash ...,"[method, dash, exact, approximate, dash, prese...","[122, 1306, 639, 655, 988]","[48, 122, 354, 360, 1005]",1.000000,"[122, 234, 1104, 924, 1307]",1.000000,"[122, 556, 1104, 924, 234]",1.000000,"[122, 234, 1104, 556, 1307]"
4,10,are real-gas transport properties for air avai...,"[real, gas, transport, property, air, availabl...","[405, 302, 436, 583, 1009]","[259, 405, 302, 436, 437]",1.000000,"[302, 185, 616, 1009, 1313]",1.000000,"[302, 332, 405, 1009, 583]",0.833333,"[302, 1009, 185, 583, 332]"


**Evaluation on train set**

In [180]:
# Finding average precision for each query
queries['tfidf_ap']=queries.apply(lambda x: avrage_precision(x['tfidf_rel'],x['ground_truth']),axis=1)

# Finding Mean Average Precision
print('Mean Average Precision=>',queries['tfidf_ap'].mean())

Mean Average Precision=> 0.655718954248366


**Evaluation on validation set**

In [182]:
# Ranking documents according to TF-IDF
validation_queries['tfidf_rel']=validation_queries['tokens'].apply(lambda x: tf_idf_rank(x))

# Finding average precision for each query
validation_queries['tfidf_ap']=validation_queries.apply(lambda x: avrage_precision(x['tfidf_rel'],x['ground_truth']),axis=1)

# Finding Mean Average Precision
print('Mean Average Precision on Validation Set=>',validation_queries['tfidf_ap'].mean())

Mean Average Precision on Validation Set=> 0.42941919191919187


#**TF-IDF BASED VECTOR SPACE MODEL**

**Creating TF-IDF vectors for Queries**

In [183]:
def gen_tfidf_queries(queries_data):
  tf_list_queries=[]

  # Getting Term frequencies
  for tokens in queries_data['tokens']:
    # Initliatizing a dictionary with 0 frequency
    queries_dict=dict.fromkeys(vocabulary,0)
    # Counting term frequencies
    for term in set(tokens).intersection(vocabulary):
      queries_dict[term]+=1
    # Adding dictionary to list
    tf_list_queries.append(queries_dict)

  # Creating a dataframe of term frequencies for queries
  queries_tf=pd.DataFrame(tf_list_queries)

  # Log Normalizing the term counts for queries
  queries_tf=queries_tf.applymap(log_normalize)

  # Calculating TF-IDF for queries
  queries_tfidf=queries_tf*list(idf_dict.values())

  # Adding a column of qids
  queries_tfidf=pd.concat([queries_data['qid'],queries_tfidf],axis=1)

  return queries_tfidf

In [184]:
# Creating TF-IDF vectors for queries in train set
queries_tfidf=gen_tfidf_queries(queries)
print('Shape=>',queries_tfidf.shape)
queries_tfidf.head()

<ipython-input-183-a701c0cd74b8>:18: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  queries_tf=queries_tf.applymap(log_normalize)


Shape=> (85, 2999)


,qid,ab,abbreviate,ability,ablate,ablating,ablation,able,abrupt,abruptly,...,year,yield,york,young,z,zbrozek,zero,zeroth,zone,zuk
0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,10,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [186]:
# Creating TF-IDF vectors for queries in validation set
queries_val_tfidf=gen_tfidf_queries(validation_queries)
print('Shape=>',queries_val_tfidf.shape)
queries_val_tfidf.head()

<ipython-input-183-a701c0cd74b8>:18: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  queries_tf=queries_tf.applymap(log_normalize)


Shape=> (22, 2999)


,qid,ab,abbreviate,ability,ablate,ablating,ablation,able,abrupt,abruptly,...,year,yield,york,young,z,zbrozek,zero,zeroth,zone,zuk
0,189,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,190,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,191,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,194,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,196,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


**Ranking**

In [187]:
# Temporary dataframe with tfidf values
temp_doc_tfidf=documents_tfidf.copy()

In [188]:
from sklearn.metrics.pairwise import cosine_similarity

In [189]:
cosine_similarity(queries_tfidf.iloc[0,1:].values.reshape(1,-1),temp_doc_tfidf.iloc[0,1:].values.reshape(1,-1))

array([[0.01603409]])

In [190]:
cosine_similarity(queries_tfidf.iloc[0,1:].values.reshape(1,-1),temp_doc_tfidf.iloc[0,1:].values.reshape(1,-1)).item()

0.016034087125217034

In [191]:
def tfidf_vsm_rank(queries_data):
  # Finding cosine similarity score for every document vector against a query vector
  temp_doc_tfidf['tfidf_vsm']=temp_doc_tfidf.apply(lambda x: cosine_similarity(x.values[1:].reshape(1, -1),queries_data[1:].values.reshape(1, -1)).item(),axis=1)

  # Sorting dataframe according to sum of cosine similarity score and getting relevant docs
  rel_docs=temp_doc_tfidf.sort_values(by='tfidf_vsm',ascending=False).head()['docid'].values

  # Droppping similarity column
  temp_doc_tfidf.drop(columns='tfidf_vsm',inplace=True)

  return rel_docs

In [192]:
# Ranking documents according to Cosine Similarity
queries['tfidf_vsm_rel']=queries_tfidf.apply(lambda x: tfidf_vsm_rank(x),axis=1)
queries.head()

,qid,query,tokens,jaccard_rel,ground_truth,avg_precision,tf_rel,tf_ap,idf_rel,idf_ap,tfidf_rel,tfidf_ap,tfidf_vsm_rel
0,1,what similarity laws must be obeyed when const...,"[similarity, law, obey, construct, aeroelastic...","[12, 51, 378, 670, 875]","[184, 29, 31, 57, 378]",0.333333,"[51, 12, 184, 364, 572]",0.333333,"[184, 51, 12, 625, 332]",1.000000,"[51, 184, 12, 332, 625]",0.50,"[184, 51, 12, 332, 875]"
1,2,what are the structural and aeroelastic proble...,"[structural, aeroelastic, problem, associate, ...","[12, 51, 700, 746, 875]","[12, 746, 15, 184, 858]",0.750000,"[12, 172, 51, 746, 798]",0.750000,"[12, 172, 51, 746, 364]",0.750000,"[12, 51, 172, 746, 724]",0.75,"[12, 51, 875, 746, 184]"
2,3,what problems of heat conduction in composite ...,"[problem, heat, conduction, composite, slab, s...","[5, 584, 6, 145, 582]","[5, 6, 90, 91, 119]",0.833333,"[5, 980, 584, 91, 395]",0.750000,"[5, 91, 625, 584, 90]",0.866667,"[5, 91, 90, 584, 625]",1.00,"[5, 90, 91, 582, 584]"
3,8,what methods -dash exact or approximate -dash ...,"[method, dash, exact, approximate, dash, prese...","[122, 1306, 639, 655, 988]","[48, 122, 354, 360, 1005]",1.000000,"[122, 234, 1104, 924, 1307]",1.000000,"[122, 556, 1104, 924, 234]",1.000000,"[122, 234, 1104, 556, 1307]",1.00,"[556, 122, 21, 569, 48]"
4,10,are real-gas transport properties for air avai...,"[real, gas, transport, property, air, availabl...","[405, 302, 436, 583, 1009]","[259, 405, 302, 436, 437]",1.000000,"[302, 185, 616, 1009, 1313]",1.000000,"[302, 332, 405, 1009, 583]",0.833333,"[302, 1009, 185, 583, 332]",1.00,"[302, 405, 583, 1009, 436]"


**Evaluation on train set**

In [193]:
# Finding average precision for each query
queries['tfidf_vsm_ap']=queries.apply(lambda x: avrage_precision(x['tfidf_vsm_rel'],x['ground_truth']),axis=1)

# Finding Mean Average Precision
print('Mean Average Precision=>',queries['tfidf_vsm_ap'].mean())

Mean Average Precision=> 0.6579575163398693


**Evaluation on Validation set**

In [194]:
# Ranking documents according to Cosine Similarity
validation_queries['tfidf_vsm_rel']=queries_val_tfidf.apply(lambda x: tfidf_vsm_rank(x),axis=1)

# Finding average precision for each query
validation_queries['tfidf_vsm_ap']=validation_queries.apply(lambda x: avrage_precision(x['tfidf_vsm_rel'],x['ground_truth']),axis=1)

# Finding Mean Average Precision
print('Mean Average Precision=>',validation_queries['tfidf_vsm_ap'].mean())

Mean Average Precision=> 0.3535353535353535


In [201]:
#@title SUMMARY
import pandas as pd

data = {
    'Method': ['Jaccard', 'TF', 'IDF', 'TF-IDF', 'TF-IDF VSM'],
    'Train Set': [0.4984, 0.5886, 0.6402, 0.6557, 0.6579],
    'Evaluation Set': [0.4454, 0.4664, 0.3800, 0.4294, 0.3535]
}

df = pd.DataFrame(data)
df

,Method,Train Set,Evaluation Set
0,Jaccard,0.4984,0.4454
1,TF,0.5886,0.4664
2,IDF,0.6402,0.3800
3,TF-IDF,0.6557,0.4294
4,TF-IDF VSM,0.6579,0.3535
